# Setting the fundarmental group of $\Sigma_{g,1}$ 

In [1]:
require './freegroup.rb'

## generators

In [2]:
genus = 3
alphabet = %w(a b c d e f)

Gens = [Letter.new] + alphabet.map{|x| Letter.new(x)}
Gens.freeze
p Gens[1..-1].map(&:show)

a = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:first)
b = [Gens[0]] + Gens[1..-1].each_slice(2).map(&:last)
p a.map(&:show)
nil

In [3]:
comms = []
(Gens.size/2 + 1).times do |i|
  comms << Group.commutator(a[i], b[i])
end
p comms.map(&:show)

["1111", "abAB", "cdCD", "efEF"]

In [4]:
Rseq = []
(Gens.size/2).times do |i|
  Rseq += [a[i+1], b[i+1].inverse, a[i+1].inverse, b[i+1]]
end
Rseq.freeze
p Rseq.map(&:show) #.index('A')
nil

<main>:2: warning: already initialized constant Rseq
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here
<main>:2: warning: already initialized constant Rseq
<main>:2: warning: previous definition of Rseq was here


["a", "B", "A", "b", "c", "D", "C", "d", "e", "F", "E", "f"]


# The Algorithm

### Wcode Class

In [5]:
class Wcode < Hash
  def initialize(word = Group::Identity)
    Rseq.each{|x| self.store(x.char, [])}
    if word != Group::Identity
      word.flatten.factors.each_with_index do |c, i|
        k = i
        self[c.char].insert(-1, 2*k)
        self[c.inverse.char].insert(0,2*k+1)
      end
    end
  end
  #---
  def show()
    self.values.map(&:to_s).join
  end
  def to_a()
    self.values.flatten
  end
  def size()
    self.to_a.size
  end
  def copy()
    wc = self.class.new
    wc.each_key{|key| wc[key] = self[key].dup}
    wc
  end
  #---
  def narrow()
    wc = self.copy
    sorted = wc.values.flatten.sort
    wc.each_key do |key|
      wc[key].map!{|v| sorted.index(v)}
    end
    wc
  end
  #---
  def to_word()
    chars = (0..self.size/2).to_a.map{|i| self.map{|key, arr| key if arr.include?(2*i)}.join}
    Word.new(chars.join)
  end
  #---
  def get_four_tuple(odd_nums)
    arr = [odd_nums[0], odd_nums[0]+1, odd_nums[1], (odd_nums[1]+1)%self.size]
    FourTuple.new(self.to_a.select{|n| arr.include?(n)}, self.size)
  end
  #---
  def devide(odd_nums)
    facs = []
    facs[0] = self.map{|key, arr| [key, arr.select{|x| x <= odd_nums[0] or odd_nums[1] < x}] }.to_h
    facs[1] = self.map{|key, arr| [key, arr.select{|x| odd_nums[0] < x and x <= odd_nums[1]}] }.to_h
    facs.map! do |f|
      wc = self.class.new
      wc.each_key{|key| wc[key] = f[key]}
    end
    return facs
  end
  #---
  def cobracket(verbose=false)
    odd_pairs=(1..self.size/2).to_a.map{|x| 2*x-1}.combination(2)
    terms = odd_pairs.map do |op|
      order = self.get_four_tuple(op)
      factors = self.devide(op)
      {order: order,
       linking: order.linking, 
       factors: factors, 
       }
    end
    return (verbose) ? terms : terms.select{|prd| prd[:linking] != 0}
  end
end; nil

#-----------------------------------
class FourTuple < Array
  def initialize(arr, mod)
    @modulo = mod
    4.times{|i| self[i] = arr[i]}
    @odd_nums = self.select{|n| n.odd?}.sort
  end
  
  def linking()
    odd1 = @odd_nums[0]
    if (self.index(odd1) - self.index(odd1+1) + 1).even?
      # [o,e,*,*] / [o,*,*,e] / [*,o,e,*] / [*,*,o,e] (and reverse 'o' and 'e')
      return 0
    else
      # [o1,e2,e1,o2] --> linking = +1 and [o2,o1,e2,e1] --> linking = +1
      # The sign of linking is change by (o1<->e1) and (e2<->o2)
      idxs = [self.index(odd1), self.index(odd1+1)] 
      sign0 = (idxs[0] < idxs[1]) ? 1 : -1
      rng = idxs.sort
      sign2 = self[rng[0]..rng[1]][1].odd? ? 1 : -1
      #---
      sign = sign0*sign2
    end
  end
end;nil

### Turaev cobraket for Word class

In [19]:
def cobracket(myw, verbose=false)  
  monomials = Wcode.new(myw).cobracket(verbose).map do |mono|
    prewords = mono[:factors].map(&:narrow).map(&:to_word)
    words = prewords.map(&:cyclic_reduce)
    mono.merge({coeff: mono[:linking], words: words})
  end
  
  result = []
  while monomials.size > 0
    pop = monomials.pop
    #---
    monomials.each do |mono|
      if mono[:words][0].conjugate?(pop[:words][0]) and mono[:words][1].conjugate?(pop[:words][1])
        pop[:coeff] += mono[:coeff]
        pop[:order] += mono[:order]
        mono[:coeff] = nil
      elsif mono[:words][0].conjugate?(pop[:words][1]) and mono[:words][1].conjugate?(pop[:words][0])
        pop[:coeff] += mono[:coeff]*(-1)
        pop[:order] += mono[:order]
        mono[:coeff] = nil
      end
    end
    #---
    if pop[:coeff] != 0 or verbose
#      result << ((pop[:coeff] == 1) ? '' : "(#{pop[:coeff].to_s}) ") + pop[:words].join(" \u{2227} ")
      result << pop
    end
    #---
    monomials.delete_if{|mono| mono[:coeff].nil?}
  end 
  return (result.empty?) ? [{orders: [], linking: 0, factors: [], coeff: 0, words: []}] : result
end; nil

# Execution

In [20]:
a1, a2 = Word.new(a[1]), Word.new(a[2])
c = (a1*Word.new(b[1])*(a1.inverse)).flatten

samples = [
  a1, 
  a1*Word.new(b[2]),
  a1*(Word.new(b[2]).inverse),
  #---
  a1^4,
  c,
  comms[1],
  #---
  a1*(comms[2].inverse),
  a1*comms[2],
  a1*(comms[2]^3),
  a1*(a2^3),
  a1*(Word.new(a[2], b[2].inverse)^3),
  a1*comms[2]*c*comms[2]*(c.inverse),
  #---
  a1*Group.commutator(a[2], a[3]),
  a1*(comms[2]*comms[3])*Group.commutator(a[2], a[3])*((comms[2]*comms[3]).inverse),
  #---
  a1*Group.commutator(b[1], a[2])
  ]

samples.each do |s|
  ws = s.show.gsub(/[()]+/, '.').gsub(/^[.]|[.]$/, '')
  cob = cobracket(s).map do |h|
    coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
    coeff + "#{h[:words].join(" \u{2227} ")}"
  end
  #---
  p "#{ws}  |--\u{03B4}-->  #{cob.join(' + ')}"
end; nil

"a  |--δ-->  (0)"
"a.d  |--δ-->  a ∧ d"
"a.D  |--δ-->  (0)"
"a.a.a.a  |--δ-->  (-1)aaa ∧ a + (-1)aa ∧ aa + (-1)a ∧ aaa"
"abA  |--δ-->  (-1)1 ∧ b"
"abAB  |--δ-->  (0)"
"a.dcDC  |--δ-->  a ∧ dcDC"
"a.cdCD  |--δ-->  (0)"
"a.cdCD.cdCD.cdCD  |--δ-->  (-1)acdCDcdCD ∧ cdCD + (-1)acdCD ∧ cdCDcdCD"
"a.c.c.c  |--δ-->  (-1)acc ∧ c + (-1)ac ∧ cc"
"a.cD.cD.cD  |--δ-->  (-1)acDcD ∧ cD + (-1)acD ∧ cDcD"
"a.cdCD.abA.cdCD.aBA  |--δ-->  (0)"
"a.ceCE  |--δ-->  aceC ∧ E + (-1)ace ∧ CE + ac ∧ C"
"a.cdCD.efEF.ceCE.feFEdcDC  |--δ-->  acdCDefEFceCfeFEdcDC ∧ E + (-1)acdCDefEFcefeFEdcDC ∧ CE + acdCDefEFcfeFEdcDC ∧ C"
"a.bcBC  |--δ-->  abcB ∧ C + (-1)abc ∧ BC + abC ∧ cB"


In [10]:
myw = samples[-1]
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true #false

wcode.cobracket(vb).each do |h|
  h.each do |k, v|
    if not k == :factors
      p "#{k}: #{v}"
    else
      p "#{k}: #{v.map(&:show).join(' , ')}" 
    end
  end
end 

printf("=====\n")
cobracket(myw, vb).each do |h|
  p "#{h[:linking]}"
  p h[:factors].map(&:show).join(" , ")
#  p h[:words].map(&:show)
  #---
  coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
  p coeff + "#{h[:words].join(" \u{2227} ")}"
  printf("----------\n")
end
nil

"(a)(bcBC)"
"[0][3, 6][1][7, 2][9, 4][][5, 8][][][][][]"
"order: [3, 1, 2, 4]"
"linking: 0"
"factors: [0][6][1][7][9, 4][][5, 8][][][][][] , [][3][][2][][][][][][][][]"
"order: [6, 1, 2, 5]"
"linking: 0"
"factors: [0][6][1][7][9][][8][][][][][] , [][3][][2][4][][5][][][][][]"
"order: [1, 7, 2, 8]"
"linking: 1"
"factors: [0][][1][][9][][8][][][][][] , [][3, 6][][7, 2][4][][5][][][][][]"
"order: [0, 1, 2, 9]"
"linking: 0"
"factors: [0][][1][][][][][][][][][] , [][3, 6][][7, 2][9, 4][][5, 8][][][][][]"
"order: [3, 6, 4, 5]"
"linking: -1"
"factors: [0][3, 6][1][7, 2][9][][8][][][][][] , [][][][][4][][5][][][][][]"
"order: [3, 7, 4, 8]"
"linking: 1"
"factors: [0][3][1][2][9][][8][][][][][] , [][6][][7][4][][5][][][][][]"
"order: [0, 3, 9, 4]"
"linking: 1"
"factors: [0][3][1][2][][][][][][][][] , [][6][][7][9, 4][][5, 8][][][][][]"
"order: [6, 7, 5, 8]"
"linking: -1"
"factors: [0][3][1][2][9, 4][][5, 8][][][][][] , [][6][][7][][][][][][][][]"
"order: [0, 6, 9, 5]"
"linking: -1"
"factors: [0]

In [21]:
myw = samples[-1]
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = true

=begin
wcode.cobracket(vb).each do |h|
  h.each do |k, v|
    if not k == :factors
      p "#{k}: #{v}"
    else
      p "#{k}: #{v.map(&:show).join(' , ')}" 
    end
  end
end 
=end

printf("=====\n")
cobracket(myw, vb).each do |h|
  h.each do |k,v|
    case k 
    when :order, :linking
      p "#{k}: #{v}"
    when :factors
      p h[:factors].map(&:show).join(" , ")
    when :words
      coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
      p coeff + "#{h[:words].join(" \u{2227} ")}"
    else
    end
  end
  printf("----------\n")
end
nil

"(a)(bcBC)"
"[0][3, 6][1][7, 2][9, 4][][5, 8][][][][][]"
=====
"order: [0, 7, 9, 8]"
"linking: 1"
"[0][3, 6][1][7, 2][4][][5][][][][][] , [][][][][9][][8][][][][][]"
"abcB ∧ C"
----------
"order: [0, 6, 9, 5]"
"linking: -1"
"[0][3][1][2][4][][5][][][][][] , [][6][][7][9][][8][][][][][]"
"(-1)abc ∧ BC"
----------
"order: [6, 7, 5, 8, 0, 3, 9, 4]"
"linking: -1"
"[0][3][1][2][9, 4][][5, 8][][][][][] , [][6][][7][][][][][][][][]"
"(0)ab ∧ B"
----------
"order: [3, 7, 4, 8]"
"linking: 1"
"[0][3][1][2][9][][8][][][][][] , [][6][][7][4][][5][][][][][]"
"abC ∧ cB"
----------
"order: [3, 6, 4, 5, 1, 7, 2, 8]"
"linking: -1"
"[0][3, 6][1][7, 2][9][][8][][][][][] , [][][][][4][][5][][][][][]"
"(0)aC ∧ c"
----------
"order: [0, 1, 2, 9]"
"linking: 0"
"[0][][1][][][][][][][][][] , [][3, 6][][7, 2][9, 4][][5, 8][][][][][]"
"(0)a ∧ bcBC"
----------
"order: [6, 1, 2, 5]"
"linking: 0"
"[0][6][1][7][9][][8][][][][][] , [][3][][2][4][][5][][][][][]"
"(0)aBC ∧ bc"
----------
"order: [3, 1, 2, 4]"
"linking:

In [9]:
myw = samples[-1] #a1*(comms[2].inverse)*a2*(Word.new(b[1]).inverse) #a1^3 #
wcode = Wcode.new(myw)
p myw.show, wcode.show

vb = false

printf("=====\n")
cobracket(myw, vb).each do |h|
  p "#{h[:linking]}"
  p h[:factors].map(&:show).join(" , ")
#  p h[:words].map(&:show)
  #---
  coeff = (h[:coeff]==1) ? '' : "(#{h[:coeff].to_s})"
  p coeff + "#{h[:words].join(" \u{2227} ")}"
  printf("----------\n")
end

nil

"(a)(bcBC)"
"[0][3, 6][1][7, 2][9, 4][][5, 8][][][][][]"
=====
"1"
"[0][3, 6][1][7, 2][4][][5][][][][][] , [][][][][9][][8][][][][][]"
"abcB ∧ C"
----------
"-1"
"[0][3][1][2][4][][5][][][][][] , [][6][][7][9][][8][][][][][]"
"(-1)abc ∧ BC"
----------
"1"
"[0][3][1][2][9][][8][][][][][] , [][6][][7][4][][5][][][][][]"
"abC ∧ cB"
----------


### TODO
$[a_{1},b_{1}]$ の処理において、
* [x] [5, 0, 6, 7] の sign が正しくないようだ。
* [x] a(x)A [ $a_{1}\otimes a_{1}^{-1}$ ] の集計もおかしい。
* [x] $a_{1}^{3}$ の計算がエラーになる。
* [ ] simple subword たちによる計算を実装する。